# Given a decision var related with time define differents constraints related with time

The constraints that can be defined are related with time for example:
- rate change. relation of a each decision variable in previous time and actual time. Ex: X(t-1) - X(t) <= rate_change
- volumen change (aka inventory change. Change of the level of the tank across the time
- fixed time t=0 in decision variables (develop in previous notebook)

**The problem is not solved in the corrected way. The codes are writted only with the idea of develop the constraints**

## Root folder and read env variables

In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-2]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## RUN

In [ ]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

## 2. Create gurobi model

In [ ]:
# create model
m = gp.Model('modelo')

### 3. Create decision variables
- Decision variables that are features
- Decicion variable that is the outputs
- Decision variable that represent the level of a tank

In [ ]:
# define set
list_set_time = ['t0', 't1', 't2', 't3', 't4', 't5', 't6']
index_set_time = pd.Index(list_set_time)
index_set_time

In [ ]:
# create decision variables - features ml model
var_X1 = gppd.add_vars(m, index_set_time, name = "decision variable Z1",
                       #lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )

var_X2 = gppd.add_vars(m, index_set_time, name = "decision variable X2",
                       #lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )

In [ ]:
# crete decision variables - output ml model

var_Y1 = gppd.add_vars(m, index_set_time, name = "decision variable Y1",
                       #lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )

var_Z1 = gppd.add_vars(m, index_set_time, name = "decision variable Y2",
                       #lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )

In [ ]:
# create decision variable that represent level of tanks
var_tank_level = gppd.add_vars(m, index_set_time, name = "decision variable level tank",
                       #lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )

### 4. Define parameters

#### 4.1 Rate change of decision variables across the time

In [ ]:
rate_change_var_X1 = 100
rate_change_var_Y1 = 100
rate_change_var_Z1 = 40

#### 4.2 Constraint tanks - lower bound and upper bound tank level

In [ ]:
tank_level_lower_bound = 200
tank_level_upper_bound = 1000

### 5. Define fixed values of the decision Var in time t=0 (actual_value) (develop in the previous notebook)
- Fix the values of period t=0 for each decision var.

- t=0 represent the actual period or initial period and it in some problems and modelations is kwown

- In addition in this notebook, the values in time t=0 are fixed for all decision variables, inclusive if the decision var have a constraint that define its values in time t = 0 (so, this kind of constraints needs to be defined since t = 1)

In [ ]:
# set values in t = 0

# var_X1['t0']
# gppd.add_constrs(m, var_X1[0], gp.GRB.EQUAL, 10, name = 'Initial Value X3') doesn't work :/
init_value_X1 = 10
m.addConstr(var_X1['t0'] == init_value_X1,  name = 'Initial Value X3')

# var_X2['t0']
init_value_X2 = 55
m.addConstr(var_X2['t0'] == init_value_X2,  name = 'Initial Value X2')


# var_Z1['t0']
init_value_Z1 = 20
m.addConstr(var_Z1['t0'] == init_value_Z1,  name = 'Initial Value Y2')


# var_tank_level['t0']
init_value_tank_level = 500
m.addConstr(var_tank_level['t0'] == init_value_tank_level,  name = 'Initial level of tank')

### 7. rate change - across time - each decision variable

\begin{align}
&| ~ X^{t}_{i} - X^{t-1}_{i} ~ | ~ \leq  ~ c_{i} &\quad \forall ~ i \in I, t \in T \tag{6}\\
\end{align}

There is a parameter between the values of a decision variable across the time. There is a delta between the actual value and the previous value

#### 7.1 Requirements constraints abs value - create auxiliar variable diff

The absolute value constraints works as:
- y >= (x1 - x2)

- y >= -(x1 - x2)

- y <= delta

**In this example**
- diff_time_x >= (x(t-1) - x(t))

- diff_time_x >= -(x(t-1) - x(t))

- diff_time_x <= delta_x

In [ ]:
###### X1
# define decision variable the difference betweeen "t" and "t-1"
diff_var_X1 = gppd.add_vars(m, index_set_time, name = "diff between X3 t-1 and t",
                            lb = -gp.GRB.INFINITY,
                            ub = gp.GRB.INFINITY
                                     )

# set initial value (diff t = 0 and t = -1) is set to cero because t = -1 is not defined
m.addConstr(diff_var_X1['t0'] == 0,  name = 'Initial Value diff X3')

In [ ]:
###### Y1
# define decision variable the difference betweeen "t" and "t-1"
diff_var_Y1 = gppd.add_vars(m, index_set_time, name = "diff between Y1 t-1 and t",
                            lb = -gp.GRB.INFINITY,
                            ub = gp.GRB.INFINITY
                                     )

# set initial value (diff t = 0 and t = -1) is set to cero because t = -1 is not defined
m.addConstr(diff_var_Y1['t0'] == 0,  name = 'Initial Value diff Y1')

In [ ]:
###### Y2
# define decision variable the difference betweeen "t" and "t-1"
diff_var_Z1 = gppd.add_vars(m, index_set_time, name = "diff between Y2 t-1 and t",
                            lb = -gp.GRB.INFINITY,
                            ub = gp.GRB.INFINITY
                                     )

# set initial value (diff t = 0 and t = -1) is set to cero because t = -1 is not defined
m.addConstr(diff_var_Z1['t0'] == 0,  name = 'Initial Value diff Y2')

#### 7.2 Define example constraints - hardocoded to one time t=1
Before automatize this constraint, it is necesary build it manually. In this example is defined for decision var X3 and time t = 1 (the difference between x3(t=0) and x3(t=1))

In [ ]:
#### example var_X1 - for time t = 1

# m.addConstr(diff_var_X1['t1'] >= (var_X1['t1'] - var_X1['t0']), name = 'diff_var_X1 positive segment')
# m.addConstr(diff_var_X1['t1'] >= -(var_X1['t1'] - var_X1['t0']), name = 'diff_var_X1 negative segment')
# m.addConstr(diff_var_X1['t1'] <= rate_change_var_X1, name = 'diff_var_X1 delta')

#### 7.3 Define full constraints - automatically all times
- Automatization constraint. For across all the set time (t0, t1, t2, etc). But it is important to see that the first value of the decision variable "diff_x" is defined before, so the "for cicle" start from (t1, t2, etc)

- This is done many times according the number of decision variables. AND the only change is the name of the decision vars. In this example, acording the names of variables, only change the name of the decision var that are defined its rate change

- So, it is necesary to genealize this codes to run only once and have a function that add this constraints

In [ ]:
### define rate change var X3

for index_time in range(1, len(index_set_time)):
    
    # define time t and t-1
    time_t = index_set_time[index_time]
    time_t_1 = index_set_time[index_time-1]
    print('time t: ', time_t)
    print('time t-1: ', time_t_1)

    # define constraints
    m.addConstr(diff_var_X1[time_t] >= (var_X1[time_t] - var_X1[time_t_1]), name = f'diff_var_X1 positive segment {time_t} - {time_t_1}')
    m.addConstr(diff_var_X1[time_t] >= -(var_X1[time_t] - var_X1[time_t_1]), name = f'diff_var_X1 negative segment {time_t} - {time_t_1}')
    m.addConstr(diff_var_X1[time_t] <= rate_change_var_X1, name = f'diff_var_X1 delta {time_t} - {time_t_1}')

In [ ]:
### define rate change var Y1

for index_time in range(1, len(index_set_time)):
    
    # define time t and t-1
    time_t = index_set_time[index_time]
    time_t_1 = index_set_time[index_time-1]
    print('time t: ', time_t)
    print('time t-1: ', time_t_1)

    # define constraints
    m.addConstr(diff_var_Y1[time_t] >= (var_Y1[time_t] - var_Y1[time_t_1]), name = f'diff_var_Y1 positive segment {time_t} - {time_t_1}')
    m.addConstr(diff_var_Y1[time_t] >= -(var_Y1[time_t] - var_Y1[time_t_1]), name = f'diff_var_Y1 negative segment {time_t} - {time_t_1}')
    m.addConstr(diff_var_Y1[time_t] <= rate_change_var_Y1, name = f'diff_var_Y1 delta {time_t} - {time_t_1}')

In [ ]:
### define rate change var Y2

for index_time in range(1, len(index_set_time)):
    
    # define time t and t-1
    time_t = index_set_time[index_time]
    time_t_1 = index_set_time[index_time-1]
    print('time t: ', time_t)
    print('time t-1: ', time_t_1)

    # define constraints
    m.addConstr(diff_var_Z1[time_t] >= (var_Z1[time_t] - var_Z1[time_t_1]), name = f'diff_var_Z1 positive segment {time_t} - {time_t_1}')
    m.addConstr(diff_var_Z1[time_t] >= -(var_Z1[time_t] - var_Z1[time_t_1]), name = f'diff_var_Z1 negative segment {time_t} - {time_t_1}')
    m.addConstr(diff_var_Z1[time_t] <= rate_change_var_Z1, name = f'diff_var_Z1 delta {time_t} - {time_t_1}')

### 8. Volumen change (Inventory change) - across time - relation between decision variables

\begin{align}
&V^{t-1}_{j} ~ + \sum_{i}  Y^{t}_{i,j} -  \sum_{i}  Z^{t}_{i',j} ~ \leq  ~ UB_{j} &\quad \forall ~ t \in T, j \in J \tag{2}\\
\end{align}

\begin{align}
&V^{t-1}_{j} ~ + \sum_{i}  Y^{t}_{i,j} -  \sum_{i}  Z^{t}_{i',j} ~ \geq  ~ LB_{j} &\quad \forall ~ t \in T, j \in J \tag{3}\\
\end{align}

\begin{align}
&V^{t-1}_{j} ~ + \sum_{i}  Y^{t}_{i,j} -  \sum_{i}  Z^{t}_{i',j} ~ =  ~ V^{t}_{j} &\quad \forall ~ t \in T, j \in J \tag{4}\\
\end{align}

#### 8.1 Define example constraints - hardocoded to one time t=1
Remember the time t = 0 is the initial time and it has fixed values

In [ ]:
# m.addConstr(var_tank_level['t0'] + var_Y1['t1'] - var_Z1['t1'] <= tank_level_upper_bound, name = 'tank level below the upper bound')
# m.addConstr(var_tank_level['t0'] + var_Y1['t1'] - var_Z1['t1'] >= tank_level_lower_bound , name = 'tank level above the lower bound')
# m.addConstr(var_tank_level['t0'] + var_Y1['t1'] - var_Z1['t1'] == var_tank_level['t1'], name = 'new level of tank')

#### 8.2 Define full constraints - automatically all times
- Automatization constraint. For across all the set time (t0, t1, t2, etc). But it is important to see that the first value of the decision variable (period t=0) is defined before, so the "for cicle" start from (t1, t2, etc)


- IMPORTANT: This codes are defined for only this example when there are only one input and one output. In the future it needs to genelize to recibe multiple input and output values

In [ ]:
for index_time in range(1, len(index_set_time)):
    
    # define time t and t-1
    time_t = index_set_time[index_time]
    time_t_1 = index_set_time[index_time-1]
    print('time t: ', time_t)
    print('time t-1: ', time_t_1)

    # define constraints
    m.addConstr(var_tank_level[time_t_1] + var_Y1[time_t] - var_Z1[time_t] <= tank_level_upper_bound, 
                name = f'tank level below the upper bound {time_t} - {time_t_1}')
    m.addConstr(var_tank_level[time_t_1] + var_Y1[time_t] - var_Z1[time_t] >= tank_level_lower_bound , 
                name = f'tank level above the lower bound {time_t} - {time_t_1}')
    m.addConstr(var_tank_level[time_t_1] + var_Y1[time_t] - var_Z1[time_t] == var_tank_level[time_t], 
                name = f'new level of tank {time_t} - {time_t_1}')

In [ ]:
m

In [ ]:
m.update()

In [ ]:
m

In [ ]:
# print show one decision var
var_X1

### 9. Additional - define a custom function as constraints that represent the relations in the process
Add a constraint defined as a funtion that represent the relation between X1, X2 with the output of the process (Y1)

In [ ]:
# define parameters of the constraint
alpha_feature_x1 = 1/5
alpha_feature_x2 = 15

In [ ]:
# define function as constraint
gppd.add_constrs(m, (alpha_feature_x1 * var_X1 + alpha_feature_x1 * var_X1), gp.GRB.EQUAL, var_Y1, name = 'function as constraint output process')

### 10. Define objective optimization
Objetive that no generate infeasibility

In [ ]:
m.setObjective(var_tank_level.sum(),
               gp.GRB.MINIMIZE)

In [ ]:
m

#### 6. Optimize and get optimal values

In [ ]:
# solve
m.optimize()

In [ ]:
#### know the status of the model - 2 a optimal solution was founded
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
m.Status

In [ ]:
# get optimal values and save in a dataframe
######## create a dataframe with set as index
solution = pd.DataFrame(index = index_set_time)

######################## save optimal values - features of models (only the features) ########################
solution["var_X1"] = var_X1.gppd.X
solution["var_X2"] = var_X2.gppd.X
solution["var_Y1"] = var_Y1.gppd.X
solution["var_Z1"] = var_Z1.gppd.X
solution["var_tank_level"] = var_tank_level.gppd.X


######################## # get value objetive function ########################
opt_objetive_function = m.ObjVal

In [ ]:
# show value objetive function
opt_objetive_function

In [ ]:
# show value decision variables
solution